## 1. Import Packages

In [1]:
from data_utils.data_utils import prepare_data, get_dataset, get_test_dataset, sound_dataset_generator, sound_dataset_generator_by_filename
from model.baseline import Baseline

import json
import numpy as np
import pandas as pd
import os
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

/home/jongwook95.lee/miniforge3/envs/hvae/lib/python3.8/site-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


## 2. Transform wav to npy.


### (1) Parameter Setting

In [2]:
experiment_result_path = "./example"
train_data_dir = "/home/jongwook95.lee/study/cnn_based_audio_classification/dataset/audio_files/train/ok/"
remove_filename_list = []
n_mels = 128
frames = 5
n_fft = 1024
hop_length = 256
power = 2
sr = 16000

batch_size = 128


### (2) Make Train Dataset

In [3]:
train_dataset_dir = os.path.join(experiment_result_path, 'train_dataset.npy')

if os.path.exists(train_dataset_dir):
    print("npy Files already exists.")
    train_dataset = np.load(train_dataset_dir)
else:
    print("Convert Audio files to Spectrogram npy files... (Train filesets)")
    train_file_list = prepare_data(train_data_dir, remove_filename_list=remove_filename_list)
    train_dataset = get_dataset(train_file_list, n_mels=n_mels, 
                                frames=frames, n_fft=n_fft, hop_length=hop_length,
                                power=power, sr=sr)
    np.save(train_dataset_dir, train_dataset) 

npy Files already exists.


### (3) Make Pytorch Dataloader (Train) 

In [4]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
train_dataset = sound_dataset_generator(train_dataset)
train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)

## 3. Model Training

### (1) Parameter Setting

In [7]:
epochs = 200
model_save_dir = "./example/best_model/baseline_model.pt"


### (2) Build Model

In [8]:
model = Baseline(input_dims = n_mels * frames)
print("number of model parameters:",sum([np.prod(p.size()) for p in model.parameters()]))
model.cuda()

number of model parameters: 267928


Baseline(
  (fc1): Linear(in_features=640, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=128, out_features=128, bias=True)
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc5): Linear(in_features=128, out_features=8, bias=True)
  (bn5): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc6): Linear(in_features=8, out_features=128, bias=True)
  (bn6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc7): Linear(in_features=128, out_features=128, bias=True)
  (bn7): BatchNorm1d(1

### (2) Set up the optimizer

In [10]:
opt = torch.optim.Adam(model.parameters(), 1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=45, gamma=0.1)
loss_fn = torch.nn.MSELoss()

In [11]:
### 5. Model Train
min_loss = 10000000
model.train()
for epoch in range(epochs):
    scheduler.step(epoch)
    loop = tqdm(train_loader, total = len(train_loader), leave = True)
    lossfs = []
    for feature in loop:
        feature = feature.float()
        feature = feature.to(device)
        out = model(feature)
        
        loss = loss_fn(out, feature)
        lossf = loss.data.item()
        lossfs.append(lossf)
        opt.zero_grad()
        loss.backward()
        opt.step()

    print("Epoch: {} / {}, train average loss: {}".format(epoch, epochs, np.mean(lossfs)))
    if min_loss > np.mean(lossfs):
        print("min loss updated {} to {}. New model saved!!".format(min_loss, np.mean(lossfs)))
        min_loss = np.mean(lossfs)
        torch.save(model.state_dict(), model_save_dir)

/home/jongwook95.lee/miniforge3/envs/hvae/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/home/jongwook95.lee/miniforge3/envs/hvae/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue

Epoch: 0 / 200, train average loss: 95.38664162965804
min loss updated 10000000 to 95.38664162965804. New model saved!!


  0%|          | 1/15750 [00:00<33:34,  7.82it/s]

Epoch: 1 / 200, train average loss: 11.34131198834616
min loss updated 95.38664162965804 to 11.34131198834616. New model saved!!


  0%|          | 1/15750 [00:00<30:34,  8.59it/s]

...

  0%|          | 1/15750 [00:00<26:54,  9.76it/s]

Epoch: 57 / 200, train average loss: 9.487691348000178
min loss updated 9.488342087881906 to 9.487691348000178. New model saved!!


 32%|███▏      | 5056/15750 [02:50<05:01, 35.45it/s] 

: 

: 

## 5. Test

### (1) Make Test Dataset

In [15]:
test_data_dir = "/home/jongwook95.lee/study/cnn_based_audio_classification/dataset/audio_files/test/ng/"

In [17]:
test_dataset_dir = os.path.join(experiment_result_path, 'test')

if os.path.exists(test_dataset_dir):
    print("npy files is already extracted.")
    test_npy_list = prepare_data(test_dataset_dir, remove_filename_list=remove_filename_list)
else:
    os.makedirs(test_dataset_dir, exist_ok=True)
    test_file_list = prepare_data(test_data_dir, remove_filename_list=remove_filename_list)
    get_test_dataset(test_file_list, test_dataset_dir, n_mels=n_mels, 
                                frames=frames, n_fft=n_fft, hop_length=hop_length,
                                power=power, sr=sr)
    test_npy_list = prepare_data(test_dataset_dir, remove_filename_list=remove_filename_list)

100%|██████████| 50/50 [00:02<00:00, 20.58it/s]


### (2) Make Pytorch Dataloader (Test) 

In [18]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
test_dataset = sound_dataset_generator_by_filename(test_npy_list)
test_loader = DataLoader(dataset=test_dataset, shuffle=False, batch_size=1)

### (3) Build Model

In [19]:
model_load_dir = "./example/best_model/baseline_model.pt"

In [20]:
model = Baseline(input_dims = n_mels * frames)
model.load_state_dict(torch.load(model_load_dir))
model.cuda()
model.eval()

Baseline(
  (fc1): Linear(in_features=640, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=128, out_features=128, bias=True)
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc5): Linear(in_features=128, out_features=8, bias=True)
  (bn5): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc6): Linear(in_features=8, out_features=128, bias=True)
  (bn6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc7): Linear(in_features=128, out_features=128, bias=True)
  (bn7): BatchNorm1d(1

### (4) Evaluate

In [21]:
anomaly_score_list = []

loop = tqdm(test_loader, total = len(test_loader), leave = True)
with torch.no_grad():
    for feature in loop:
        feature = feature.float()
        feature = feature[0]
        feature = feature.to(device)
        out = model(feature)
        loss = loss_fn(out, feature)

        loss_average = torch.mean(loss).cpu().detach()
        loss_median = torch.median(loss).cpu().detach()
        loss_list = [loss_average, loss_median]
        loss = np.array([loss_list])
        anomaly_score_list = anomaly_score_list + [loss]

100%|██████████| 50/50 [00:00<00:00, 302.92it/s]


### (5) Save a Result

In [22]:
test_prediction_filename = 'baseline_test_ng'

In [23]:
anomaly_score_list = np.vstack(anomaly_score_list)
result1 = pd.DataFrame(anomaly_score_list)
file_name_list = [os.path.basename(file) for file in test_npy_list]
result = pd.DataFrame({'File': file_name_list})
result = pd.concat([result, result1], axis = 1)
result.columns = ['File', 'Mean', 'Median']
result_path = os.path.join(experiment_result_path, 'test_prediction')
os.makedirs(result_path, exist_ok=True)
result.to_csv(result_path + '/' + test_prediction_filename + '.csv', index = False)